# 📊 Visualização de Dados: Dataset de Indicadores de Diabetes

Este notebook demonstra como criar visualizações poderosas usando **matplotlib.pyplot** e **seaborn** com o dataset de indicadores de saúde para diabetes, e como integrá-las no **Streamlit**.

## 🎯 Objetivos:
- Explorar diferentes tipos de gráficos com matplotlib
- Entender padrões nos dados de diabetes
- Demonstrar integração com Streamlit
- Comparar visualizações estáticas vs interativas

## 📈 Dataset: BRFSS 2015 Diabetes Health Indicators
- **Registros**: ~70.000 pacientes reais
- **Features**: 21 indicadores de saúde
- **Target**: Predição de diabetes (0/1)

## 1️⃣ Import Required Libraries

In [ ]:
# Import bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Para usar no Streamlit
# import streamlit as st

print("✅ Bibliotecas importadas com sucesso!")
print("📊 Matplotlib version:", plt.matplotlib.__version__)
print("🎨 Seaborn version:", sns.__version__)
print("📈 Plotly version:", px.__version__)

## 2️⃣ Load and Explore Dataset

In [ ]:
# Carregar dataset de diabetes
df = pd.read_csv('data/diabetes_binary_5050split_health_indicators_BRFSS2015.csv')

print("📊 Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Colunas: {list(df.columns)}")
print("\n📈 Distribuição do Target:")
print(df['Diabetes_binary'].value_counts())
print(f"Percentual com diabetes: {df['Diabetes_binary'].mean():.1%}")

# Informações básicas
print("\n🔍 Informações do Dataset:")
df.info()

# Estatísticas descritivas
print("\n📊 Estatísticas Descritivas:")
df.describe().round(2)

## 3️⃣ Visualizações com Matplotlib

In [ ]:
### 📊 1. Distribuição de BMI por Status de Diabetes

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Histograma
axes[0].hist(df[df['Diabetes_binary'] == 0]['BMI'], alpha=0.7, label='Sem Diabetes', bins=30, color='lightblue')
axes[0].hist(df[df['Diabetes_binary'] == 1]['BMI'], alpha=0.7, label='Com Diabetes', bins=30, color='salmon')
axes[0].set_xlabel('BMI (Índice de Massa Corporal)')
axes[0].set_ylabel('Frequência')
axes[0].set_title('Distribuição de BMI por Status de Diabetes')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
box_data = [df[df['Diabetes_binary'] == 0]['BMI'], df[df['Diabetes_binary'] == 1]['BMI']]
axes[1].boxplot(box_data, labels=['Sem Diabetes', 'Com Diabetes'])
axes[1].set_ylabel('BMI')
axes[1].set_title('Box Plot: BMI por Status de Diabetes')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Para usar no Streamlit:
# st.pyplot(fig)
# plt.clf()  # Limpar figura após usar no Streamlit

In [ ]:
### 🔥 2. Heatmap de Correlação entre Variáveis

# Calcular matriz de correlação
correlation_matrix = df.corr()

# Criar heatmap
plt.figure(figsize=(16, 12))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdYlBu_r', 
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={"shrink": .8})

plt.title('Matriz de Correlação - Indicadores de Saúde para Diabetes', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# Identificar correlações mais fortes com diabetes
diabetes_corr = correlation_matrix['Diabetes_binary'].sort_values(key=abs, ascending=False)
print("🎯 Correlações mais fortes com Diabetes:")
print(diabetes_corr.drop('Diabetes_binary').head(10))

In [ ]:
### 📈 3. Gráficos de Barras: Fatores de Risco

# Preparar dados para gráficos de barras
categorical_vars = ['HighBP', 'HighChol', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity']
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, var in enumerate(categorical_vars):
    # Calcular percentuais de diabetes para cada categoria
    diabetes_rates = df.groupby(var)['Diabetes_binary'].agg(['count', 'mean'])
    
    # Criar gráfico de barras
    axes[i].bar(diabetes_rates.index, diabetes_rates['mean'] * 100, 
                color=['lightblue', 'salmon'], alpha=0.8)
    axes[i].set_title(f'Taxa de Diabetes por {var}', fontsize=12)
    axes[i].set_xlabel(f'{var} (0=Não, 1=Sim)')
    axes[i].set_ylabel('Taxa de Diabetes (%)')
    axes[i].grid(True, alpha=0.3)
    
    # Adicionar valores no topo das barras
    for j, rate in enumerate(diabetes_rates['mean'] * 100):
        axes[i].text(j, rate + 1, f'{rate:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
### 🎻 4. Violin Plots: Distribuições Detalhadas

# Variáveis contínuas interessantes
continuous_vars = ['BMI', 'MentHlth', 'PhysHlth', 'Age']
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, var in enumerate(continuous_vars):
    # Separar dados por status de diabetes
    no_diabetes = df[df['Diabetes_binary'] == 0][var]
    with_diabetes = df[df['Diabetes_binary'] == 1][var]
    
    # Criar violin plot
    parts = axes[i].violinplot([no_diabetes, with_diabetes], positions=[0, 1], showmeans=True)
    
    # Personalizar cores
    for pc, color in zip(parts['bodies'], ['lightblue', 'salmon']):
        pc.set_facecolor(color)
        pc.set_alpha(0.7)
    
    axes[i].set_xticks([0, 1])
    axes[i].set_xticklabels(['Sem Diabetes', 'Com Diabetes'])
    axes[i].set_ylabel(var)
    axes[i].set_title(f'Distribuição de {var} por Status de Diabetes')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4️⃣ Integração com Streamlit

In [ ]:
### 💡 Como usar matplotlib no Streamlit

def create_streamlit_visualization():
    """
    Exemplo de como integrar matplotlib no Streamlit
    """
    # Código para usar no Streamlit:
    """
    import streamlit as st
    import matplotlib.pyplot as plt
    import pandas as pd
    
    # Função para criar gráfico matplotlib
    def plot_bmi_distribution(df):
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Histograma
        ax.hist(df[df['Diabetes_binary'] == 0]['BMI'], 
                alpha=0.7, label='Sem Diabetes', bins=30, color='lightblue')
        ax.hist(df[df['Diabetes_binary'] == 1]['BMI'], 
                alpha=0.7, label='Com Diabetes', bins=30, color='salmon')
        
        ax.set_xlabel('BMI')
        ax.set_ylabel('Frequência')
        ax.set_title('Distribuição de BMI por Status de Diabetes')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        return fig
    
    # No Streamlit app:
    st.title("📊 Análise de Diabetes")
    
    # Carregar dados
    df = load_data()
    
    # Criar gráfico
    fig = plot_bmi_distribution(df)
    
    # Exibir no Streamlit
    st.pyplot(fig)
    
    # IMPORTANTE: Limpar figura após usar
    plt.clf()  # ou plt.close(fig)
    """
    
print("💡 Exemplo de código para Streamlit criado!")
print("📋 Principais pontos:")
print("1. Use st.pyplot(fig) para exibir matplotlib")
print("2. Sempre use plt.clf() ou plt.close() após st.pyplot()")
print("3. Crie funções separadas para cada gráfico")
print("4. Considere usar st.cache_data para performance")

In [ ]:
### 🔥 Exemplo Prático: Dashboard de Análise

def create_dashboard_example():
    """
    Exemplo completo de dashboard com matplotlib + Streamlit
    """
    streamlit_code = '''
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Configurar página
st.set_page_config(
    page_title="Dashboard Diabetes",
    page_icon="🏥",
    layout="wide"
)

# Sidebar para filtros
st.sidebar.header("🔧 Filtros")
age_range = st.sidebar.slider("Faixa Etária", 1, 13, (1, 13))
bmi_range = st.sidebar.slider("BMI", 10.0, 100.0, (10.0, 100.0))

# Título principal
st.title("🏥 Dashboard de Análise de Diabetes")

# Layout em colunas
col1, col2 = st.columns(2)

with col1:
    st.subheader("📊 Distribuição por Idade")
    
    # Gráfico 1: Distribuição por idade
    fig1, ax1 = plt.subplots(figsize=(8, 5))
    
    # Dados filtrados
    filtered_data = df[(df['Age'] >= age_range[0]) & 
                      (df['Age'] <= age_range[1]) &
                      (df['BMI'] >= bmi_range[0]) & 
                      (df['BMI'] <= bmi_range[1])]
    
    # Contar por idade e diabetes
    age_diabetes = filtered_data.groupby(['Age', 'Diabetes_binary']).size().unstack(fill_value=0)
    
    age_diabetes.plot(kind='bar', ax=ax1, color=['lightblue', 'salmon'])
    ax1.set_title('Casos de Diabetes por Faixa Etária')
    ax1.set_xlabel('Faixa Etária')
    ax1.set_ylabel('Número de Casos')
    ax1.legend(['Sem Diabetes', 'Com Diabetes'])
    
    st.pyplot(fig1)
    plt.close(fig1)

with col2:
    st.subheader("🎯 Fatores de Risco")
    
    # Gráfico 2: Fatores de risco
    fig2, ax2 = plt.subplots(figsize=(8, 5))
    
    risk_factors = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack']
    diabetes_rates = []
    
    for factor in risk_factors:
        rate = filtered_data[filtered_data[factor] == 1]['Diabetes_binary'].mean() * 100
        diabetes_rates.append(rate)
    
    bars = ax2.bar(risk_factors, diabetes_rates, color='coral')
    ax2.set_title('Taxa de Diabetes por Fator de Risco (%)')
    ax2.set_ylabel('Taxa de Diabetes (%)')
    ax2.tick_params(axis='x', rotation=45)
    
    # Adicionar valores nas barras
    for bar, rate in zip(bars, diabetes_rates):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{rate:.1f}%', ha='center', va='bottom')
    
    st.pyplot(fig2)
    plt.close(fig2)

# Gráfico de correlação em linha completa
st.subheader("🔥 Mapa de Correlação")

fig3, ax3 = plt.subplots(figsize=(12, 8))

# Selecionar apenas colunas numéricas
numeric_cols = filtered_data.select_dtypes(include=[np.number]).columns
correlation_matrix = filtered_data[numeric_cols].corr()

# Heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, ax=ax3, fmt='.2f')
ax3.set_title('Matriz de Correlação - Indicadores de Saúde')

st.pyplot(fig3)
plt.close(fig3)

# Métricas importantes
st.subheader("📈 Métricas Importantes")

col3, col4, col5, col6 = st.columns(4)

with col3:
    total_cases = len(filtered_data)
    st.metric("Total de Casos", total_cases)

with col4:
    diabetes_rate = filtered_data['Diabetes_binary'].mean() * 100
    st.metric("Taxa de Diabetes", f"{diabetes_rate:.1f}%")

with col5:
    avg_bmi = filtered_data['BMI'].mean()
    st.metric("BMI Médio", f"{avg_bmi:.1f}")

with col6:
    high_risk = filtered_data[(filtered_data['HighBP'] == 1) & 
                             (filtered_data['HighChol'] == 1)]['Diabetes_binary'].mean() * 100
    st.metric("Risco Alto (%)", f"{high_risk:.1f}%")
'''
    
    print("🔥 Código completo do dashboard criado!")
    print("\n📋 Recursos inclusos:")
    print("• Filtros interativos (idade, BMI)")
    print("• Gráficos matplotlib integrados")
    print("• Layout responsivo com colunas")
    print("• Métricas em tempo real")
    print("• Mapa de correlação")
    print("• Limpeza automática de figuras")
    
    return streamlit_code

# Executar exemplo
dashboard_code = create_dashboard_example()

## 5️⃣ Próximos Passos

### ✅ O que já temos funcionando:
- ✅ Sistema completo de predição de diabetes
- ✅ Interface em português
- ✅ Upload e processamento de dados
- ✅ Treinamento automatizado
- ✅ Métricas de performance
- ✅ API FastAPI + Streamlit + PostgreSQL

### 🚀 Melhorias possíveis com matplotlib:

1. **Dashboard de Análise** 📊
   - Gráficos interativos de distribuição
   - Análise de correlações
   - Visualização de fatores de risco

2. **Relatórios Médicos** 📋
   - Gráficos para profissionais de saúde
   - Comparações por demografia
   - Tendências temporais

3. **Análise Exploratória** 🔍
   - Boxplots por grupos
   - Scatter plots multivariados
   - Histogramas comparativos

4. **Performance do Modelo** 📈
   - Curvas ROC interativas
   - Matriz de confusão visual
   - Feature importance plots

### 💡 Dicas de implementação:
- Use `st.pyplot()` para matplotlib
- Sempre chame `plt.close()` após exibir
- Considere `st.cache_data` para datasets grandes  
- Combine com `st.selectbox()` para filtros interativos

In [ ]:
### 🎯 Resumo Final

print("🏥 ANÁLISE COMPLETA DE VISUALIZAÇÕES PARA DIABETES")
print("=" * 60)

print("\n✅ O que exploramos:")
print("• Distribuições de BMI por diabetes")
print("• Mapas de correlação entre indicadores")
print("• Análise de fatores de risco")  
print("• Gráficos de violino para comparações")
print("• Integração completa com Streamlit")

print("\n🔧 Tecnologias utilizadas:")
print("• matplotlib.pyplot - Gráficos básicos")
print("• seaborn - Visualizações estatísticas")
print("• streamlit - Interface web interativa")
print("• pandas - Manipulação de dados")

print("\n💡 Principais aprendizados:")
print("• matplotlib funciona perfeitamente no Streamlit")
print("• Use st.pyplot(fig) para exibir gráficos")
print("• Sempre limpe figuras com plt.close()")
print("• Combine com filtros interativos para maior valor")

print("\n🚀 Próximo passo sugerido:")
print("Implementar uma página 'Análise de Dados' no sistema existente")
print("com gráficos matplotlib para análise exploratória dos dados!")

print("\n" + "=" * 60)
print("🎉 Notebook de análise completo!")
print("📊 Pronto para implementar visualizações no sistema!")